In [ ]:
import pickle 
import pandas as pd
import spacy
from tqdm.auto import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.notebook import tqdm
tqdm.pandas()
import re
import transformers
from sentence_transformers import SentenceTransformer, util
import nltk
from nltk import tokenize
import pickle
import urllib.request as requests
import json
import statistics
import torch
from collections import defaultdict
import copy
import math

In [ ]:
nltk.download('punkt')
nlp = spacy.load("en_core_web_lg")

model = SentenceTransformer('all-mpnet-base-v2')

df=pd.read_csv('gmo_to_uniqueId.csv')
materialID=df['lastRunJob'].to_list()

with open('PreProcessed_segmented_spacy.pkl', 'rb') as f:
    newmetalist=pickle.load(f)

sampled_datadf=pd.read_csv("Sampled_Adword_Labeled.csv")

#For Normal process with target labels only

df1=pd.read_csv('InitialTargetDatabase.csv')    
target_dict=dict(zip(df1['Name'], df1.index))

In [ ]:
def clean(text):
    text=text.replace("(", "")
    text=text.replace(")", "")
    text=text.replace(";", "")
    text=text.replace(",", "")
    text=text.replace("+", "")
    text=text.replace(".", "")
    return text

In [ ]:
def process_text(text):
    #doc = self.nlp(unicode(text.lower()))
    text=text.replace('\\n', ' ')
    doc = nlp(text.lower())
    result = []
    for token in doc:
        if token.text in nlp.Defaults.stop_words:
            continue
        if token.is_punct:
            continue
        if token.lemma_ == '-PRON-':
            continue
        result.append(token.lemma_)
    return nlp(" ".join(result))

In [ ]:
print(df1.head())

In [ ]:
df1['Combined']=df1['Name'] #+' '+df1['info']
df1['Combined1']=df1['Name'] +' '+df1['info']
df1['Combined1'].apply(clean)
df1['Combined1'].apply(process_text)
df1['Combined'].apply(clean)
targetlist=df1['Combined'].to_list()
targetlistcalc=df1['Combined1'].to_list()

In [75]:
temp=targetlistcalc[27]
temp=temp.replace('*News business use case needs to be scoped by NBCU - priority TBD', '')
targetlistcalc[27]=temp

In [ ]:
print(targetlistcalc)

In [ ]:
truthdatadf=pd.read_csv('AdwordLabeledDatabase.csv')
truthdatadf.head()

In [ ]:
def getadword(episode, truthdatadf):
    for i in range(len(truthdatadf.lastRunJob)):
        if episode == truthdatadf.lastRunJob[i]:
            return truthdatadf.adword[i]

In [ ]:
sampled_datadf.head()

In [ ]:
listofepisodes=sampled_datadf['lastRunJob'].to_list()
len(listofepisodes)

In [ ]:
#listofepisodes=['FILE_MAF_20220211T205601Z_GMO_00000000001313_01', 'FILE_MAF_20220211T205607Z_GMO_00000000001534_01', 'FILE_MAF_20210727T231031Z_GMO_00000000001577_01' ]

In [ ]:
def keydict(dataframe, targetlist):
    d=defaultdict(list)
    for target in targetlist:
        minidf=dataframe.loc[dataframe['adword']==target]
        minilist=minidf['lastRunJob'].to_list()
        d[target]=minilist
    
    return d



In [ ]:
targetkey_filelist=keydict(sampled_datadf, targetlist)
print(targetkey_filelist)



In [ ]:
# #NRS bulk with top 1 save to file regular
# dict_to_save=defaultdict(list)
# for target in tqdm(targetlist):
#     scores=[]
#     for ep in tqdm(listofepisodes):
#         index=materialID.index(ep)
#         groundtruth=getadword(ep,truthdatadf)
#         # if groundtruth in target_dict:
#         #         x=target_dict[groundtruth]
#         x=target_dict[target]
#         #print(groundtruth)
#         print(x)
#         segmentscore=[]
#         for i in range(len(newmetalist[index])):
#             if len(newmetalist[index][i])==0:
#                 continue
#             emb1=model.encode(newmetalist[index][i]) #, convert_to_tensor=True)
#             emb2=model.encode(targetlist) #, convert_to_tensor=True)
#             cos_sim=util.cos_sim(emb1,emb2)

#             #For OG Class
#             MaxAggregation = {}
#             for i in range(cos_sim.shape[1]):
#                 all_sentence_combinations=[]
#                 for j in range(cos_sim.shape[0]):
#                     all_sentence_combinations.append([cos_sim[j][i], j, i])
#                 all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)
#                 MaxAggregation[i]=all_sentence_combinations[0]
                

#             segmentscore.append(MaxAggregation[x][0])

            
                    
#         avg=(sum(segmentscore)/len(segmentscore))
#         scores.append(avg)
        
#     print(scores)
#     dict_to_save[target]=scores

In [ ]:
# with open('NRS_top1_regular.pkl', 'wb') as f:
#     pickle.dump(dict_to_save, f)


In [ ]:
# with open('NRS_top1_regular.pkl', 'rb') as f:
#     NRS_dict=pickle.load(f)


In [ ]:
def get_key(val):
    for key, value in target_dict.items():
         if val == value:
             return key
 
    return "key doesn't exist"

In [65]:
#Relevance calculation per episode 
dictofrelevance=copy.deepcopy(targetkey_filelist)
for key, value in tqdm(targetkey_filelist.items()): #for each IAB Label
    if len(targetkey_filelist[key])==0:
        continue
    for i in tqdm(range(len(targetkey_filelist[key]))): #for each File
        index=materialID.index(targetkey_filelist[key][i])
        x=target_dict[key] #Index of the target label
        MaxAggregation=defaultdict(list)
        segmentscore=[]
        for l in range(len(newmetalist[index])): #For each Segment
            res=[]
            for j in range(len(targetlist)): 
            # print(len(newmetalist[index]))
                if len(newmetalist[index][l])==0: 
                    continue
                org_doc=nlp(newmetalist[index][l])
                doc_ref = nlp(targetlist[j])
                sim_score = org_doc.similarity(doc_ref)
                MaxAggregation[j].append(sim_score)

        
        for k,v in MaxAggregation.items():
            temp=MaxAggregation[k]
            avg=sum(temp)/len(temp)
            MaxAggregation[k]=[avg]

            

        dictofrelevance[key][i]={dictofrelevance[key][i]:MaxAggregation}
        print(dictofrelevance[key][i])

with open('Relevance_spaCy.pkl', 'wb') as f:
    pickle.dump(dictofrelevance, f)   
 
        #RS and NRS for OG Class

        # RSlist=MaxAggregation[x]
        # score=[]
        # for s in RSlist:
        #     score.append(s[0])

        # rs=sum(score)/len(score)

        # NRSlist=NRS_dict[key]
        # nrsdenom=sum(NRSlist)-rs/len(NRSlist)

        # NRS=rs/nrsdenom

        
        # #RS and ClassPredicted for Predicted Class

        # order=[]
        # for key in MaxAggregation.keys():
        #     l=MaxAggregation[key]
        #     summ=[]
        #     for val in l:
        #         summ.append(val[0])
        #     order.append(sum(summ))
        
        
        # max_val=max(order)
        # max_index=order.index(max_val)
        # ClassPredicted=get_key(max_index)

        # RSlistss=MaxAggregation[max_index]
        # scoress=[]
        # for s in RSlistss:
        #     scoress.append(s[0])

        # RS_CP=sum(scoress)/len(scoress)

        # d={targetkey_filelist[key][i]:{'NRS': NRS, "Class_Predicted": ClassPredicted, "RS_CP": RS_CP, "RS_OG": rs }}

        


        


        

    

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

{'FILE_MAF_20211223T065128Z_GMO_00000000002584_01': defaultdict(<class 'list'>, {0: [0.2846420821618714], 1: [0.5084100619287447], 2: [0.5667064033525411], 3: [0.4165372078670444], 4: [0.4730639735007759], 5: [0.5926549105877715], 6: [0.5423939547713682], 7: [0.5379312694661945], 8: [0.5842983651466362], 9: [0.365499795719492], 10: [0.505724915179389], 11: [0.5210131978908443], 12: [0.3556484796548685], 13: [0.5355455108623679], 14: [0.39198800250430643], 15: [0.5025801872075861], 16: [0.48514501446694724], 17: [0.3691274213007689], 18: [0.3916692238422676], 19: [0.3944957991462822], 20: [0.3763743393591347], 21: [0.4181226569366336], 22: [0.3496140407595095], 23: [0.33813712353585135], 24: [0.4144102637545056], 25: [0.4307941091399359], 26: [0.3707818117921535], 27: [0.6017141643156709], 28: [0.4955453606083624]})}
{'FILE_MAF_20211223T064615Z_GMO_00000000106986_01': defaultdict(<class 'list'>, {0: [0.2564839358061105], 1: [0.5102700815712914], 2: [0.5684022473046803], 3: [0.4117861667

  0%|          | 0/20 [00:00<?, ?it/s]

{'FILE_MAF_20210729T092828Z_GMO_00000000044326_01': defaultdict(<class 'list'>, {0: [0.302094632470782], 1: [0.5368559233426394], 2: [0.6163432642151554], 3: [0.4455991412401147], 4: [0.4917070975105494], 5: [0.6098709958569011], 6: [0.5592483743103333], 7: [0.5006619859327361], 8: [0.5503205008609313], 9: [0.3972194461042494], 10: [0.5070652100989197], 11: [0.5096974591824226], 12: [0.3728635177398498], 13: [0.5341539516873747], 14: [0.36496163728520076], 15: [0.48066757562814877], 16: [0.5351948967107963], 17: [0.3757798103544038], 18: [0.4148199657922736], 19: [0.42120953478422046], 20: [0.4077865870441516], 21: [0.41786590262762546], 22: [0.3568559879353891], 23: [0.35670203240897524], 24: [0.44323237657954284], 25: [0.4531047149211732], 26: [0.38999748889951713], 27: [0.6136585464815884], 28: [0.5602662697862477]})}
{'FILE_MAF_20210730T234421Z_GMO_00000000044076_01': defaultdict(<class 'list'>, {0: [0.23943006134527423], 1: [0.5117788904163159], 2: [0.5408128773061349], 3: [0.3806

...

{'FILE_MAF_20220213T011912Z_GMO_00000000022391_01': defaultdict(<class 'list'>, {0: [0.2649545494150429], 1: [0.5231928644153248], 2: [0.5865909748274069], 3: [0.418958372334344], 4: [0.485395649956177], 5: [0.6167069213396281], 6: [0.5606694191889167], 7: [0.5310471604485104], 8: [0.5809489035633846], 9: [0.4015848115218104], 10: [0.501496344973536], 11: [0.48207553257952646], 12: [0.3969427474799145], 13: [0.55803919735458], 14: [0.37988749215833056], 15: [0.5259909135900681], 16: [0.5090012649799359], 17: [0.40328460778511616], 18: [0.4046902089655213], 19: [0.4132333672376091], 20: [0.3895071029748177], 21: [0.44348151732320357], 22: [0.368699499447157], 23: [0.3547129969067369], 24: [0.4164143840849841], 25: [0.46926983620232204], 26: [0.3790022177980407], 27: [0.6256226389446014], 28: [0.5213356207250239]})}
{'FILE_MAF_20210809T184702Z_GMO_00000000171498_01': defaultdict(<class 'list'>, {0: [0.23230823591598979], 1: [0.5080766972518592], 2: [0.5389729873214866], 3: [0.39036130279

  0%|          | 0/20 [00:00<?, ?it/s]

{'FILE_MAF_20210727T231034Z_GMO_00000000001600_01': defaultdict(<class 'list'>, {0: [0.2381011870863067], 1: [0.5175420115187307], 2: [0.5287934960916124], 3: [0.3896465137859069], 4: [0.4733112891263082], 5: [0.6089077528686294], 6: [0.5752927106638319], 7: [0.5592251223167496], 8: [0.5747998088780848], 9: [0.3857193600630131], 10: [0.5197283395530589], 11: [0.4364746249757865], 12: [0.4167941046017753], 13: [0.543842384052304], 14: [0.37347008302725837], 15: [0.5302191302616457], 16: [0.49743640769977127], 17: [0.4169335864897523], 18: [0.3839748057207291], 19: [0.4170850982183976], 20: [0.39102906969032714], 21: [0.4609744260501558], 22: [0.29421178293961897], 23: [0.3366577668735659], 24: [0.40408306744053685], 25: [0.44448569476508926], 26: [0.3490495302914525], 27: [0.5900694349151973], 28: [0.4696262938656015]})}
{'FILE_MAF_20210730T044004Z_GMO_00000000040639_01': defaultdict(<class 'list'>, {0: [0.24994129116261224], 1: [0.5140636225579445], 2: [0.5496268940373142], 3: [0.40430

  0%|          | 0/20 [00:00<?, ?it/s]

{'FILE_MAF_20220213T185059Z_GMO_00000000044111_01': defaultdict(<class 'list'>, {0: [0.22904233371242966], 1: [0.5095193770616248], 2: [0.533333444117181], 3: [0.3770320898965147], 4: [0.47360123248563574], 5: [0.5921457851735709], 6: [0.5737696748891484], 7: [0.5550078785109152], 8: [0.5716569067085054], 9: [0.38918655302824473], 10: [0.5048635758968892], 11: [0.43217474920800947], 12: [0.40818098075466774], 13: [0.5278933623532559], 14: [0.3688051820239023], 15: [0.5137945554285864], 16: [0.4740979555704234], 17: [0.38653505953399003], 18: [0.37875691542139256], 19: [0.41283666810533043], 20: [0.3952608454009133], 21: [0.4720847692480487], 22: [0.3104612796503064], 23: [0.3426243364296062], 24: [0.40688893224192824], 25: [0.4427421274673848], 26: [0.3497552075420158], 27: [0.5948498636796565], 28: [0.46513090017568776]})}
{'FILE_MAF_20220215T035715Z_GMO_00000000320875_01': defaultdict(<class 'list'>, {0: [0.24473145406395047], 1: [0.49046969003457325], 2: [0.5363579961360135], 3: [0.

In [66]:
#Def Top1 Agg
def segmentcombiner(dic): #for segments to final dic of relevance scores
    dic1=copy.deepcopy(dic)
    filename=list(dic1.keys())[0]
    defdict=dic1[filename]
    for k,v in defdict.items():
        RSlist=defdict[k]
        score=[]
        for s in RSlist:
            score.append(s)

        rs=sum(score)/len(score)
        defdict[k]=rs
    d={filename:defdict}
    return d
    


In [ ]:
#Def Top3 and Harm Agg
# def segmentcombinertop3Harm(dic): #for segments to final dic of relevance scores
#     dic1=copy.deepcopy(dic)
#     filename=list(dic1.keys())[0]
#     defdict=dic1[filename]
#     for k,v in defdict.items():
#         RSlist=defdict[k][0]
#         score=[]
#         for s in RSlist:
#             score.append(s[0])

#         rs=sum(score)/len(score)
#         defdict[k]=rs
#     d={filename:defdict}
#     return d

In [67]:
with open('Relevance_spaCy.pkl', 'rb') as f:
     Relevance_dict=pickle.load(f)

In [68]:
print(NRS(Relevance_dict))

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

...

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

{'Automotive': 1.074038484602269, 'Books and Literature': 1.0297294456284134, 'Business and Finance': 1.0294534045651886, 'Education': 1.0383823700784724, 'Events and Attractions': 0.9897245097385086, 'Family and Relationships': 1.020467127031697, 'Food & Drink': 1.0992088178377681, 'Hobbies & Interests': 1.0323040557555037, 'Medical Health': 1.144651004203355, 'Music and Audio': 1.0279652463025812, 'Pets': 1.0005492535541172, 'Real Estate': 1.1719038634778423, 'Religion & Spirituality': 1.043469253778164, 'Science': 1.026414654084103, 'Shopping': 1.0406821904512675, 'Sports': 1.0527029012489688, 'Style & Fashion': 1.232470932671084, 'Technology & Computing': 1.117047931610644, 'Television': 1.0458622111807014, 'Travel': 1.0197023609194096}
None


In [50]:
#Def NRS for top1 
def NRS(dict):
    d={}
    for k,v in tqdm(dict.items()):#for each label
        if len(dict[k]) == 0:
            continue
        d1={}
        for target in tqdm(targetlist): #for each topic
            scoreforfiles=[]
            if len(dict[target]) == 0:
                continue
            for i in range(len(dict[k])): #for each file in label
                x=target_dict[target]
                combinedsegments=segmentcombiner(dict[k][i])
                key=list(combinedsegments.keys())[0]
                scoreforfiles.append(float(combinedsegments[key][x])) #combine segments 
            avg=sum(scoreforfiles)/len(scoreforfiles)
            d1[target]=avg
        d[k]=d1

    finaldict={}
    for k,v in d.items():
        numerator=d[k][k]
        denominator=[]
        for kk,vv in d.items():
            denominator.append(d[kk][k])
        avgdenominator=(sum(denominator)-numerator)/(len(denominator)-1)
        NRS=numerator/avgdenominator
        finaldict[k]=NRS
    print(finaldict)


            
        

In [69]:
print(RSMetrics(Relevance_dict))

  0%|          | 0/29 [00:00<?, ?it/s]

{'Automotive': {'RS15': 100.0, 'RS20': 100.0, 'RS35': 0.0}, 'Books and Literature': {'RS15': 100.0, 'RS20': 100.0, 'RS35': 100.0}, 'Business and Finance': {'RS15': 100.0, 'RS20': 100.0, 'RS35': 100.0}, 'Education': {'RS15': 100.0, 'RS20': 100.0, 'RS35': 100.0}, 'Events and Attractions': {'RS15': 100.0, 'RS20': 100.0, 'RS35': 100.0}, 'Family and Relationships': {'RS15': 100.0, 'RS20': 100.0, 'RS35': 100.0}, 'Food & Drink': {'RS15': 100.0, 'RS20': 100.0, 'RS35': 100.0}, 'Hobbies & Interests': {'RS15': 100.0, 'RS20': 100.0, 'RS35': 100.0}, 'Medical Health': {'RS15': 100.0, 'RS20': 100.0, 'RS35': 100.0}, 'Music and Audio': {'RS15': 100.0, 'RS20': 100.0, 'RS35': 100.0}, 'Pets': {'RS15': 100.0, 'RS20': 100.0, 'RS35': 100.0}, 'Real Estate': {'RS15': 100.0, 'RS20': 100.0, 'RS35': 100.0}, 'Religion & Spirituality': {'RS15': 100.0, 'RS20': 100.0, 'RS35': 100.0}, 'Science': {'RS15': 100.0, 'RS20': 100.0, 'RS35': 100.0}, 'Shopping': {'RS15': 100.0, 'RS20': 100.0, 'RS35': 100.0}, 'Sports': {'RS15':

In [70]:
#Def RS at 0.15, 0.20, 0.35 and Mean RS 
def RSMetrics(dict):
    d={}
    for k,v in tqdm(dict.items()): #for each label
        if len(dict[k]) == 0:
            continue
        scoreforfiles=[]
        for i in range(len(dict[k])): #for each file in label
            x=target_dict[k]
            combinedsegments=segmentcombiner(dict[k][i])
            key=list(combinedsegments.keys())[0]
            scoreforfiles.append(round(float(combinedsegments[key][x]),2))
        rs15=[]
        rs20=[]
        rs35=[]
        for item in scoreforfiles:
            if item >= 0.15:
                rs15.append(item)
            if item >= 0.20:
                rs20.append(item)
            if item >= 0.35:
                rs35.append(item)

        RS15=(len(rs15)/len(scoreforfiles))*100
        RS20=(len(rs20)/len(scoreforfiles))*100
        RS35=(len(rs35)/len(scoreforfiles))*100
        d1={"RS15":RS15, "RS20": RS20, "RS35": RS35}
        d[k]=d1
    
    return d
        
    


In [71]:
print(Prediction(Relevance_dict))

  0%|          | 0/29 [00:00<?, ?it/s]

{'Automotive': {'FILE_MAF_20211223T065128Z_GMO_00000000002584_01': {'ClassPredicted': 'News and Politics', 'RS_CP': 0.6017141643156709}, 'FILE_MAF_20211223T064615Z_GMO_00000000106986_01': {'ClassPredicted': 'Family and Relationships', 'RS_CP': 0.6163317611842932}, 'FILE_MAF_20211223T064819Z_GMO_00000000002709_01': {'ClassPredicted': 'News and Politics', 'RS_CP': 0.6056331133723035}, 'FILE_MAF_20211222T231607Z_GMO_00000000294982_01': {'ClassPredicted': 'News and Politics', 'RS_CP': 0.6088405691841746}, 'FILE_MAF_20211222T214634Z_GMO_00000000029298_01': {'ClassPredicted': 'Family and Relationships', 'RS_CP': 0.5976846186763813}, 'FILE_MAF_20211222T214941Z_GMO_00000000121162_01': {'ClassPredicted': 'Family and Relationships', 'RS_CP': 0.6017906276099642}, 'FILE_MAF_20211222T230805Z_GMO_00000000273733_01': {'ClassPredicted': 'News and Politics', 'RS_CP': 0.6007698180894656}, 'FILE_MAF_20211223T065125Z_GMO_00000000027759_01': {'ClassPredicted': 'News and Politics', 'RS_CP': 0.61277304573440

In [72]:
#Def Predicted Class and RS of Class
def Prediction(dict):
    d={}
    for k,v in tqdm(dict.items()): #for each label
        if len(dict[k]) == 0:
            continue
        d1={}
        for i in range(len(dict[k])): #for each file in label
            combinedsegments=segmentcombiner(dict[k][i])
            key=list(combinedsegments.keys())[0]
            order=[]
            for kk in combinedsegments[key].keys():
                l=combinedsegments[key][kk]
                order.append(l)
            max_val=max(order)
            max_index=order.index(max_val)
            ClassPredicted=get_key(max_index)
            RS_CP=combinedsegments[key][max_index]
            d2={'ClassPredicted': ClassPredicted, 'RS_CP': RS_CP }
            d1[key]=d2
        d[k]=d1

    return d

            # 
            # scoreforfiles.append(float(combinedsegments[key][x]0))

            # order=[]
            # for key in MaxAggregation.keys():
            #     l=MaxAggregation[key]
            #     summ=[]
            #     for val in l:
            #         summ.append(val[0])
            #     order.append(sum(summ))
            
            
            # max_val=max(order)
            # max_index=order.index(max_val)
            # ClassPredicted=get_key(max_index)

            # RSlistss=MaxAggregation[max_index]
            # scoress=[]
            # for s in RSlistss:
            #     scoress.append(s[0])

            # RS_CP=sum(scoress)/len(scoress)
    

In [73]:
print(mAP(Relevance_dict))

  0%|          | 0/29 [00:00<?, ?it/s]

({'Automotive': 0.0, 'Books and Literature': 0.0, 'Business and Finance': 0.0, 'Education': 0.0, 'Events and Attractions': 0.0, 'Family and Relationships': 95.0, 'Food & Drink': 60.0, 'Hobbies & Interests': 0.0, 'Medical Health': 0.0, 'Music and Audio': 20.0, 'Pets': 0.0, 'Real Estate': 0.0, 'Religion & Spirituality': 0.0, 'Science': 0.0, 'Shopping': 0.0, 'Sports': 0.0, 'Style & Fashion': 0.0, 'Technology & Computing': 0.0, 'Television': 0.0, 'Travel': 0.0}, 8.75)


In [74]:
#Def mAP
def mAP(dic):
    res=Prediction(dic)
    d={}
    mAP=[]
    for k,v in res.items():
        classes=[]
        truth=[]

        for kk,vv in res[k].items():
            ClassPred=res[k][kk]['ClassPredicted']
            if ClassPred==k:
                classes.append(ClassPred)
            truth.append(k)
        precision=(len(classes)/len(truth))*100
        mAP.append(precision)
        d[k]=precision
    mAPfinal=sum(mAP)/len(mAP)
    return(d,mAPfinal)
    

